* Description of the function, the arguments and the returns
* Author information, links to PLOS One paper

### Input:
    * timeseries
    * winsize (default = 50)
    * detrending (no, gaussian, loess, linear, first-diff)
    * bandwidth (default=NULL)
    * span (NULL)
    * degree (NULL)
    * logtransform (FALSE)
    * interpolate (FALSE) 
    * AR_n (FALSE)
    * powerspectrum (FALSE)
    
## example function description
def cubic_roots(a, b, c, d):

    """Compute the roots of the cubic polynomial :math:`ax^3 + bx^2 + cx + d`.
    
    :param a: cubic coefficient
    :param b: quadratic coefficient
    :param c: linear coefficient
    :param d: constant
    :return: list of three complex roots
    
    This function does not check if the found roots are real or complex.
    """

def generic_ews(timseries, winsize = 50, detrending = c("no", "gaussian", 
    "loess", "linear", "first-diff"), bandwidth = NULL, span = NULL, degree = NULL, 
    logtransform = FALSE, interpolate = FALSE, AR_n = FALSE, powerspectrum = FALSE):
    
    
    
###  Check timeseries (Bregje) + Unit-test (Pablo)
    * input: timeindex and data (different than Dakos)
    * make it a np array
    * check if timeindex and data are same length
    * check if timeindex is included, otherwise generate timeindex with the length of the timeseries
    * rename the timeseries with the data as Y
    * warning if not the correct format
    * warning if not evenly spaced


    
### Interpolation
    * find python equivalent of 'approx' to interpolate
    * if-loop (if interpolate)
    * interpolate timeseries and assign new timeseries to Y
    * else: Y=Y
    (not included in our version)
    
### Log-transformation (Ingrid)
    * if logtransform, transform Y+1 using a python equivalent of log 
    * look up function in Python
    * multivariate 
    
### Detrending (Arie)
    * assign argument of detrending in the function to variable detrending
    * if detrending == 'gaussian'
         * if bandwidth is NULL
             * define pre-set bandwidth (Vasilis uses a specific function for this, see article PLOS one)
         * else
             bandwidth = round(length of timeseries * bandwidth /100)
         * smooth the timeseries using equivalent of ksmooth in python
         * assign residuals to nsmY and smoothed ts to smY
    * if detrending  == linear
        * assign residuals of linear fit to nsmY
        * assign fit to smY
    * if detrending == loess:
        * if span not defined:
            * span = 25/100
        * else
            span = span/100
        * if degree not defined:
            degree=2
        * else
            degree = degree
        * apply loess to data (family = gaussian, no normalization)
        * smY = predict using results of loess
        * nsmY = time series - smY #(residuals)
    * else if detrending == firstdiff:
        * nsmY  is difference between two consecutive time points 
        * define new timeindex as timeindexdiff
        
        * no default bandwidth --> for later
    
## Function  EWS (Els)
* params need to be which indicators should be calculated e.g. autocorrelation=True, variance=True etc
* default indicators = False
* not on running window (don't look too much at the R code)

### Calculate indicators
    * create variables (in R Vasilis uses numeric())
        * nARR, nSD, nSK, nKURT, nACF, nDENSITYRATIO, nCV, smARall, smARmaxeig, detB, ARn
        * and matrix nSPECT (matrix(0, nrow=mw, ncol=ncol(nMR)))
    * calculate standard deviation of nMR and assign to nSD (remove na)
    * for loop i in range ncol(nMR):
        * fit autoregressive model to data (nMR[:,i]) and assign to the variable nYR (don't apply Akaike Information Criterion, maximum order of model fit is 1, AR model should be for  x minus its mean and do not fit a separate intercept term)
        * assign estimated autoregression coefficients for the fitted model to nARR[i]
        * nSK[i] is skewness of that part of the data
        * assign kurtosis of that part of the data to nKURT[i] 
        * nCV[i] = nSD[i]/mean(nMR[:,i])
        * assign estimate of the autocorrelation function to ACF (find equivalent of acf-function in python and equivalent of 'acf' output of R function to nACF[i]
        * compute estimate of spectra density from AR fit 
        * assign spectrum to nSPECT[:,i]
        * compute densitiyratio as follows: spectrum[low]/spectrum[high] and assign to nDENSITYRATIO[i]
    * if AR_n is defined:
        * Calculate resilience indicators based on AR(n) as in paper of Ives 2003
        * fit autoregressive model to data and apply akaike information criterion, maximum order is 6, AR model should not be for x minus its mean and do not fit a sperate intercept term. Call variable ARall
        * assign ar[1] to smARall[i]
        * calculate equivalent of:  roots <- Mod(polyroot(c(rev(-ar), 1))) in python
        * assing max(roots) to smARmaxeig[i]
        * detB[i] = (prod(roots))^(2/ARn[i])
    * nRETURNRATE = 1/nARR


## Function EWS Rolling window (Ingrid )
### Rearrange data for indicator calculation
    * define the following variables:
        * mw, the moving window (round(length(Y) *winsize/100))
        * omw, the number of moving windows (length(nsmY) - mw +1)
        * low = 2 #
        * high = mw
        * nMR, a matrix to store data of different moving windows (matrix(data=NA, nrow=mw, ncol=omw)
        * x1, timindex of one mowing window (1:mw)
    * for-loop (i in range moving windows): # so: (1 in 1:omw), to apply moving window to Y data
        * Ytw = nsmY[i:(i + mw -1)]  # generate Y data with length of moving window
        * nMR[:,i] = Ytw # store in nMR
        
   * use function EWS
    

    
    
### Estimate Kendall trend statistic for indicators (Jelle)
    * timevec = sequence 1:length(nARR)
    * find equivalent of cor.test() in python
    * use Kendall and two-sided t-test with onfience level of 0.95
    * determine Kendall trend as follows:
        * KtAR = test of nARR
        * KtACF is test of nACF
        * KtSD = test of nSD
        * KtSK is test of nSK
        * KtKU is test of nKURT
        * KtDENSITYRATIO = test of nDENSITYRATIO
        * KrRETURNRATE  is test of nRETURNRATE
        * KtCV is test of nCV
        
* drawbacks of this method?
* Carl Boettiger 
 
    
### Plotting Generic Early-Warnings
    * open a new device 
    * set settings for plotting
    * plot timeindex, Y
    * plot the smoothed line, depending on which method was chosen
    * plot nARR, nACF, nRETURNRATE, nDENSITYRATIO, nSD, nSK, nKURT
    * see code vasilis for exact format 
    
    
### Resilience Estimators based on AR(n)
    * if AR_n is TRue, plot the resilience estimators based on AR(n)
    * plot ARn, smARmaxeig, detB, smARall

    
### Power spectrum
    * if powerspectrum 
        * open new device
        * set settings for plotting
        * make contourplot of frequency
        
    
### Output
    * out = a dataframe with the following columns: timeindex, nARR, nSD, nSK, nKURT, nCV, nRETURNRATE, nDENSITYRATIO, nACF and no row names
    * assign columnnames to the dataframe
    * return out


### Test logtransform

In [13]:
import numpy as np
ts=np.random.rand(2,10)
ts_log = np.log(ts+1)

### Test rolling window

    * define the following variables:
        * mw, the moving window (round(length(Y) *winsize/100))
        * omw, the number of moving windows (length(nsmY) - mw +1)
        * low = 2 #
        * high = mw
        * nMR, a matrix to store data of different moving windows (matrix(data=NA, nrow=mw, ncol=omw)
        * x1, timindex of one mowing window (1:mw)
    * for-loop (i in range moving windows): # so: (1 in 1:omw), to apply moving window to Y data
        * Ytw = nsmY[i:(i + mw -1)]  # generate Y data with length of moving window
        * nMR[:,i] = Ytw # store in nMR
        
   * use function EWS

In [64]:
ts=np.random.rand(10)
winsize=50

# WE SHOULD CHECK HERE THAT ts is one-dimensional.. 

mw=round(ts.size * winsize/100) # length moving window
omw=ts.size-mw+1 # number of moving windows
nMR = np.empty(shape=(omw,mw))
nMR[:] = np.nan

#not needed in this function: 
low=2 
high=mw 
x = range(1,mw) 

for i in range(0,omw):
    nMR[i,:]=ts[i:i+mw]  
    

[ 0.75309008  0.28057337  0.85606325  0.90607309  0.6030068   0.07881576
  0.80744605  0.84650663  0.81879282  0.94718915]
5
6
[[ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]]
